# Generate graph for multiple benign and attacked imgs

## Import packages

In [5]:
# Set auto reload for python functions
%reload_ext autoreload
%autoreload 2

# Helper libraries
import os
import cv2
import glob
import tqdm
import numpy as np
from time import time
import tensorflow as tf
import matplotlib.pyplot as plt
from networkx.readwrite import json_graph

# GoogLeNet
import lucid.modelzoo.vision_models as models

# Libraries provided by Massif project
import constant
import A_matrix
import I_matrix
import model_helper
import gen_graph

print(tf.VERSION)

1.15.0


## Import GoogLeNet (InceptionV1) model

In [4]:
googlenet = models.InceptionV1()
googlenet.load_graphdef()

## Get constants and hyperparameters

In [6]:
args = constant.Args
arg_keys = [arg for arg in args.__dict__.keys() if '__' not in arg]

print('Constants and hyperparameters in args:')
print(arg_keys)

Constants and hyperparameters in args:
['gpu', 'batch_A', 'batch_I', 'layer', 'k_A', 'layer_sizes', 'layers', 'layer_blk_sizes', 'blk_headers', 'num_classes', 'img_width', 'img_height']


## Read benign images

In [8]:
benign_dirpath = '../data/sample-images/sample-benign'

benign_imgs = []
for benign_filename in os.listdir(benign_dirpath):
    if 'jpg' in benign_filename:
        benign_img = plt.imread('{}/{}'.format(benign_dirpath, benign_filename))
        benign_imgs.append(benign_img)

## Read attacked images with different strength

In [19]:
attacked_dirpath = '../data/sample-images/sample-attacked'
targeted_classname = 'armadillo'

attacked_imgs = {}
for attacked_filename in os.listdir(attacked_dirpath):
    if targeted_classname not in attacked_filename:
        continue
    if 'npy' in attacked_filename:
        split_str_in_filename = attacked_filename.split('-')
        eps = float(split_str_in_filename[-2])
        if eps not in attacked_imgs:
            attacked_imgs[eps] = []
        img = np.load('{}/{}'.format(attacked_dirpath, attacked_filename))
        attacked_imgs[eps].append(img)

## A_matrix: generate nodes

- A-matrix: shape of (#layers, #neurons)

### A_matrix of the **benign** images

In [30]:
A_benign = A_matrix.init_A_matrix_single_class(args)

In [33]:
act_scores_benign = model_helper.get_all_layers_activation_score(googlenet, benign_imgs, args.layers)

In [51]:
for layer in args.layers:
    median_act_score_across_imgs = np.median(act_scores_benign[layer], axis=0)
    A_benign[layer] = median_act_score_across_imgs

### A_matrix of the **attacked** image

In [56]:
epss = [2, 2.5, 3, 3.5]

A_attacked_dict = {}
for eps in epss:
    A_attacked = A_matrix.init_A_matrix_single_class(args)
    A_attacked_dict[eps] = A_attacked

In [57]:
for eps in epss:
    
    # Get the activation scores
    act_scores_attacked = model_helper.get_all_layers_activation_score(googlenet, attacked_imgs[eps], args.layers)
    
    # Save the median activation scores
    for layer in args.layers:
        median_act_score_across_imgs = np.median(act_scores_attacked[layer], axis=0)
        A_attacked_dict[eps][layer] = median_act_score_across_imgs

## I_matrix: generate edges

- I_matrix for a given layer: (#neurons, #previous_neurons)

### I_matrix of the **benign** image

In [20]:
Is_benign = I_matrix.gen_I_matrices(args, [raw_benign], googlenet)

In [21]:
I_benign = Is_benign[0]

### I_matrix of the **attacked** image

In [23]:
Is_attacked = I_matrix.gen_I_matrices(args, [raw_attacked], googlenet)

In [24]:
I_attacked = Is_attacked[0]

### I_matrix of the **correct** class (white wolf) and **misclassified** class (mexican hairless)

In [22]:
# Directory Path
I_mat_summit_dir_path = '/Users/haekyu/data/summit/I-mat'

# Read and parse the summit I matrices
tic = time()
I_correct_class = gen_graph.I_from_summit_to_massif(args, I_mat_summit_dir_path, white_wolf)
I_mis_class = gen_graph.I_from_summit_to_massif(args, I_mat_summit_dir_path, mexican_hairless)
toc = time()
print('%.2lf sec' % (toc - tic))

167.26 sec


### Prepare vis

In [26]:
got_activated = {}
got_idle = {}
both_activated = {}

for layer in args.layers:
    benign_activations = As_benign[layer]
    attacked_activations = As_attacked[layer]

    diff = benign_activations - attacked_activations
    abs_diff = np.abs(diff)
    sorted_neurons_benign = np.argsort(benign_activations)
    sorted_neurons_attacked = np.argsort(attacked_activations)
    sorted_neurons_diff = np.argsort(diff)

    rank_of_neurons_attacked = sorted_neurons_attacked[::-1]
    rank_of_neurons_benign = sorted_neurons_benign[::-1]

    got_activated_neurons, no_longer_activated_neurons, both_activated_neurons = [], [], []
    for neuron in rank_of_neurons_attacked:

        rank_in_attacked = np.argwhere(rank_of_neurons_attacked == neuron)
        rank_in_benign = np.argwhere(rank_of_neurons_benign == neuron)

        is_responsive_to_attacked = rank_in_attacked < top_rank_thresold
        is_responsive_to_attacked = is_responsive_to_attacked[0][0]

        is_non_responsive_to_benign = rank_in_benign > low_rank_threshold
        is_non_responsive_to_benign = is_non_responsive_to_benign[0][0]

        is_non_responsive_to_attacked = rank_in_attacked > low_rank_threshold
        is_non_responsive_to_attacked = is_non_responsive_to_attacked[0][0]

        is_responsive_to_benign = rank_in_benign < top_rank_thresold
        is_responsive_to_benign = is_responsive_to_benign[0][0]

        if is_responsive_to_attacked and is_non_responsive_to_benign:
            got_activated_neurons.append(neuron)

        if is_non_responsive_to_attacked and is_responsive_to_benign:
            no_longer_activated_neurons.append(neuron)

        if is_responsive_to_attacked and is_responsive_to_benign:
            both_activated_neurons.append(neuron)

    got_activated[layer] = got_activated_neurons[:]
    got_idle[layer] = no_longer_activated_neurons[:]
    both_activated[layer] = both_activated_neurons[:]

print('got activated')
print(got_activated)
print('\ngot_idle')
print(got_idle)
print('\nboth_activated')
print(both_activated)

got activated
{'mixed3a': [], 'mixed3b': [309, 133, 379, 158, 15, 380], 'mixed4a': [390], 'mixed4b': [229, 367, 307], 'mixed4c': [236, 245, 50, 412], 'mixed4d': [330, 341], 'mixed4e': [6, 763, 25, 620, 389, 783, 280], 'mixed5a': [732, 814, 394, 739, 407, 499, 9, 760, 307, 207, 496], 'mixed5b': [854, 973, 32, 236, 524, 788, 64, 474, 841, 886, 930, 775, 888, 402, 417, 129, 229, 783, 400, 436, 197, 851, 303, 427, 654, 667, 346]}

got_idle
{'mixed3a': [], 'mixed3b': [362, 345, 370, 152], 'mixed4a': [326, 396, 244, 232, 392], 'mixed4b': [176, 375], 'mixed4c': [321, 166, 178], 'mixed4d': [37, 168, 326], 'mixed4e': [423, 339, 732, 660, 73, 816, 818, 765], 'mixed5a': [688, 513, 293, 58, 445, 768, 413, 239, 265], 'mixed5b': [447, 358, 921, 225, 91, 522, 902, 335, 941, 12, 175, 915, 643, 735, 340, 195, 11, 857, 523, 101, 476, 289]}

both_activated
{'mixed3a': [190, 143, 67, 69, 157, 102, 184, 122, 71, 81, 152, 206, 117, 169, 133, 96, 173, 179, 101, 154, 156, 127, 105, 124, 137, 185, 196, 92, 146

## Generate full graph

### Generate full graph of the **Benign** image

In [ ]:
G_benign = gen_graph.gen_full_graph(args, As_benign, I_benign)

In [ ]:
G_benign_json = json_graph.node_link_data(G_benign)

In [ ]:
benign_str_parsed_nodes = list(map(lambda x: {'weight': str(x['weight']), 'id': x['id']}, G_benign_json['nodes']))
benign_str_parsed_links = list(map(lambda x: {'source': x['source'], 'target': x['target'], 'weight': str(x['weight'])}, G_benign_json['links']))
G_benign_str_json = {'nodes': benign_str_parsed_nodes, 'links': benign_str_parsed_links}

In [ ]:
graph_dir_path = '/Users/haekyu/data/massif/full-graph'
file_path = '{}/{}.json'.format(graph_dir_path, 'G-benign')
with open(file_path, 'w') as f:
    json.dump(G_benign_str_json, f)

### Generate full graph of the **Attacked** image

In [ ]:
G_attacked = gen_graph.gen_full_graph(args, As_attacked, I_attacked)

In [ ]:
G_attacked_json = json_graph.node_link_data(G_attacked)

In [ ]:
attacked_str_parsed_nodes = list(map(lambda x: {'weight': str(x['weight']), 'id': x['id']}, G_attacked_json['nodes']))
attacked_str_parsed_links = list(map(lambda x: {'source': x['source'], 'target': x['target'], 'weight': str(x['weight'])}, G_attacked_json['links']))
G_attacked_str_json = {'nodes': attacked_str_parsed_nodes, 'links': attacked_str_parsed_links}

In [ ]:
graph_dir_path = '/Users/haekyu/data/massif/full-graph'
file_path = '{}/{}.json'.format(graph_dir_path, 'G-attacked')
with open(file_path, 'w') as f:
    json.dump(G_attacked_str_json, f)

### Generate full graph of the **correct** class (white wolf)

In [ ]:
tic = time()
G_correct = gen_graph.gen_full_graph(args, As_correct_class, I_correct_class)
toc = time()
print('Generate full graph of the correct class: %.2lf sec' % (toc - tic))

In [ ]:
tic = time()
G_correct_json = json_graph.node_link_data(G_correct)
toc = time()
print('Full graph into json format: %.2lf sec' % (toc - tic))

In [ ]:
tic = time()
correct_str_parsed_nodes = list(map(lambda x: {'weight': str(x['weight']), 'id': x['id']}, G_correct_json['nodes']))
correct_str_parsed_links = list(map(lambda x: {'source': x['source'], 'target': x['target'], 'weight': str(x['weight'])}, G_correct_json['links']))
G_correct_str_json = {'nodes': correct_str_parsed_nodes, 'links': correct_str_parsed_links}
toc = time()
print('Json str parsing: %.2lf sec' % (toc - tic))

In [ ]:
tic = time()
graph_dir_path = '/Users/haekyu/data/massif/full-graph'
file_path = '{}/{}.json'.format(graph_dir_path, 'G-correct')
with open(file_path, 'w') as f:
    json.dump(G_correct_str_json, f)
toc = time()
print('Save graph: %.2lf sec' % (toc - tic))

### Generate full graph of the **misclassified** class (mexican hairless)

In [ ]:
tic = time()
G_mis = gen_graph.gen_full_graph(args, As_mis_class, I_mis_class)
toc = time()
print('Generate full graph of the misclassified class: %.2lf sec' % (toc - tic))

In [ ]:
tic = time()
G_mis_json = json_graph.node_link_data(G_mis)
toc = time()
print('Full graph into json format: %.2lf sec' % (toc - tic))

In [ ]:
tic = time()
mis_str_parsed_nodes = list(map(lambda x: {'weight': str(x['weight']), 'id': x['id']}, G_mis_json['nodes']))
mis_str_parsed_links = list(map(lambda x: {'source': x['source'], 'target': x['target'], 'weight': str(x['weight'])}, G_mis_json['links']))
G_mis_str_json = {'nodes': mis_str_parsed_nodes, 'links': mis_str_parsed_links}
toc = time()
print('Json str parsing: %.2lf sec' % (toc - tic))

In [ ]:
tic = time()
graph_dir_path = '/Users/haekyu/data/massif/full-graph'
file_path = '{}/{}.json'.format(graph_dir_path, 'G-mis')
with open(file_path, 'w') as f:
    json.dump(G_mis_str_json, f)
toc = time()
print('Save graph: %.2lf sec' % (toc - tic))

### Generate full graph of a class (General)

In [ ]:
import json 

# Directory Path
A_mat_summit_dir_path = '/Users/haekyu/data/summit/A-mat'
I_mat_summit_dir_path = '/Users/haekyu/data/summit/I-mat'
graph_dir_path = '/Users/haekyu/data/massif/full-graph'
    
for c in range(200):
    print('class:', c)

    # Read A matrices
    tic = time()
    A_class = A_matrix.init_A_matrix_single_image(args)
    for layer in args.layers:
        A_mat_path = '{}/A-0.03-{}.csv'.format(A_mat_summit_dir_path, layer)
        A_mat = np.loadtxt(A_mat_path, delimiter=',')
        A_class[layer] = A_mat[c]
    toc = time()
    print('Read A: %.2lf sec' % (toc - tic))
    
    # Read and parse the summit I matrices
    tic = time()
    I_class = gen_graph.I_from_summit_to_massif(args, I_mat_summit_dir_path, c)
    toc = time()
    print('Read I: %.2lf sec' % (toc - tic))
    
    tic = time()
    G_class = gen_graph.gen_full_graph(args, A_class, I_class)
    toc = time()
    print('Generate full graph: %.2lf sec' % (toc - tic))

    tic = time()
    G_class_json = json_graph.node_link_data(G_class)
    toc = time()
    print('Full graph into json format: %.2lf sec' % (toc - tic))

    tic = time()
    class_str_parsed_nodes = list(map(lambda x: {'weight': str(x['weight']), 'id': x['id']}, G_class_json['nodes']))
    class_str_parsed_links = list(map(lambda x: {'source': x['source'], 'target': x['target'], 'weight': str(x['weight'])}, G_class_json['links']))
    G_class_str_json = {'nodes': class_str_parsed_nodes, 'links': class_str_parsed_links}
    toc = time()
    print('Json str parsing: %.2lf sec' % (toc - tic))

    tic = time()
    file_path = '{}/{}.json'.format(graph_dir_path, 'G-{}'.format(c))
    with open(file_path, 'w') as f:
        json.dump(G_class_str_json, f)
    toc = time()
    print('Save graph: %.2lf sec' % (toc - tic))

class: 0
Read A: 2.43 sec
Read I: 107.49 sec
Generate full graph: 189.60 sec
Full graph into json format: 5.99 sec
Json str parsing: 3.03 sec
Save graph: 30.85 sec
class: 1
Read A: 3.56 sec
Read I: 148.17 sec
Generate full graph: 204.08 sec
Full graph into json format: 9.49 sec
Json str parsing: 4.69 sec
Save graph: 24.08 sec
class: 2
Read A: 2.94 sec
Read I: 133.85 sec


284.2105263157895

## Compare the results of benign and attacked images -- ex) layer = mixed4c

In [19]:
layer = 'mixed4d'
benign_activations = As_benign[layer]
attacked_activations = As_attacked[layer]

diff = benign_activations - attacked_activations
abs_diff = np.abs(diff)
sorted_neurons_benign = np.argsort(benign_activations)
sorted_neurons_attacked = np.argsort(attacked_activations)
sorted_neurons_diff = np.argsort(diff)

In [ ]:
print('Compare activation, neurons sorted by benign')
print('Ignore the x-axis numbers; they are not neurons exactly.')
plt.plot(benign_activations[sorted_neurons_benign])
plt.plot(attacked_activations[sorted_neurons_benign])
plt.plot(diff[sorted_neurons_benign])
plt.legend(['Benign', 'Attacked', 'Difference (Benign - Attacked)'])
plt.xlabel('Neurons sorted by benign')
plt.ylabel('Activation')
plt.show()

In [ ]:
print('Compare activation, neurons sorted by attacked')
print('Ignore the x-axis numbers; they are not neurons exactly.')
plt.plot(benign_activations[sorted_neurons_attacked])
plt.plot(attacked_activations[sorted_neurons_attacked])
plt.plot(diff[sorted_neurons_attacked])
plt.legend(['Benign', 'Attacked', 'Difference (Benign - Attacked)'])
plt.xlabel('Neurons sorted by attacked')
plt.ylabel('Activation')
plt.show()

In [ ]:
print('Compare activation, neurons sorted by difference between attacked and benign')
print('Ignore the x-axis numbers; they are not neurons exactly.')

plt.figure()
reverse_sorted_neurons = sorted_neurons_diff[::-1]
reverse_sorted_activation = np.sort(diff)[::-1]
plt.plot(reverse_sorted_activation)
plt.ylabel("activation difference")
plt.xlabel('Neurons sorted by difference between attacked and benign')
plt.show()
print('<--- More benign side      ...     More attacked side --->')

## Find out neurons influenced by the attack
- The neurons are one of the **highly** activated ones for **attacked** image and **misclassified** class.
- The neurons are one of the **barely** activated ones for **benign** image and **original** class.

In [20]:
topk = 10
top_standard = 0.1 # 10%
low_standard = 0.3 # 30%

num_total_neurons = len(sorted_neurons_attacked)
top_rank_thresold = num_total_neurons * top_standard
low_rank_threshold = num_total_neurons * low_standard

In [21]:
rank_of_neurons_attacked = sorted_neurons_attacked[::-1]
rank_of_neurons_benign = sorted_neurons_benign[::-1]

got_activated_neurons, no_longer_activated_neurons, both_activated_neurons = [], [], []
for neuron in rank_of_neurons_attacked:
    
    rank_in_attacked = np.argwhere(rank_of_neurons_attacked == neuron)
    rank_in_benign = np.argwhere(rank_of_neurons_benign == neuron)
    
    is_responsive_to_attacked = rank_in_attacked < top_rank_thresold
    is_responsive_to_attacked = is_responsive_to_attacked[0][0]
    
    is_non_responsive_to_benign = rank_in_benign > low_rank_threshold
    is_non_responsive_to_benign = is_non_responsive_to_benign[0][0]
    
    is_non_responsive_to_attacked = rank_in_attacked > low_rank_threshold
    is_non_responsive_to_attacked = is_non_responsive_to_attacked[0][0]
    
    is_responsive_to_benign = rank_in_benign < top_rank_thresold
    is_responsive_to_benign = is_responsive_to_benign[0][0]
    
    if is_responsive_to_attacked and is_non_responsive_to_benign:
        got_activated_neurons.append(neuron)
    
    if is_non_responsive_to_attacked and is_responsive_to_benign:
        no_longer_activated_neurons.append(neuron)
    
    if is_responsive_to_attacked and is_responsive_to_benign:
        both_activated_neurons.append(neuron)
        

In [ ]:
print('Look into neurons that got activatied by the attacked image, while was idle by benign image.')
print('The attacked image newly got the following features to fool the model.')
print('Neurons:', got_activated_neurons)
print()

vis_dirpath = '/Users/haekyu/data/summit/summit-data/data/feature-vis'
for neuron in got_activated_neurons:
    
    rank_in_attacked = np.argwhere(rank_of_neurons_attacked == neuron)
    rank_in_benign = np.argwhere(rank_of_neurons_benign == neuron)
    top_percent_in_attacked = 100 * rank_in_attacked / num_total_neurons
    top_percent_in_benign = 100 * rank_in_benign / num_total_neurons
    
    print('{}-{}'.format(layer, neuron))
    print('Top %.2lf%% in benign (White wolf) --> top %.2lf%% in attacked (Mexican hairless)' % (top_percent_in_benign, top_percent_in_attacked))
    
    plt.figure()
    
    # Feature vis
    plt.subplot(231)
    feature_vis_path = '{}/channel/{}-{}-channel.jpg'.format(vis_dirpath, layer, neuron)
    feature_vis = plt.imread(feature_vis_path)
    plt.axis('off')
    plt.imshow(feature_vis)
    
    # Example patch 0
    plt.subplot(234)
    ex0_path = '{}/dataset-p/{}-{}-dataset-p-0.jpg'.format(vis_dirpath, layer, neuron)
    ex0 = plt.imread(ex0_path)
    plt.axis('off')
    plt.imshow(ex0)
    
    # Example patch 1
    plt.subplot(232)
    ex1_path = '{}/dataset-p/{}-{}-dataset-p-1.jpg'.format(vis_dirpath, layer, neuron)
    ex1 = plt.imread(ex1_path)
    plt.axis('off')
    plt.imshow(ex1)
    
    # Example patch 2
    plt.subplot(235)
    ex2_path = '{}/dataset-p/{}-{}-dataset-p-2.jpg'.format(vis_dirpath, layer, neuron)
    ex2 = plt.imread(ex2_path)
    plt.axis('off')
    plt.imshow(ex2)
    
    # Example patch 3
    plt.subplot(233)
    ex3_path = '{}/dataset-p/{}-{}-dataset-p-3.jpg'.format(vis_dirpath, layer, neuron)
    ex3 = plt.imread(ex3_path)
    plt.axis('off')
    plt.imshow(ex3)

    # Example patch 4
    plt.subplot(236)
    ex4_path = '{}/dataset-p/{}-{}-dataset-p-4.jpg'.format(vis_dirpath, layer, neuron)
    ex4 = plt.imread(ex4_path)
    plt.axis('off')
    plt.imshow(ex4)
    
    plt.show()


In [ ]:
print('Look into neurons that are no longer activated in attacked image, while was very activated by benign image.')
print('The attacked images lose the following features.')
print('Neurons:', no_longer_activated_neurons)
print()

vis_dirpath = '/Users/haekyu/data/summit/summit-data/data/feature-vis'
for neuron in no_longer_activated_neurons:
    
    rank_in_attacked = np.argwhere(rank_of_neurons_attacked == neuron)
    rank_in_benign = np.argwhere(rank_of_neurons_benign == neuron)
    top_percent_in_attacked = 100 * rank_in_attacked / num_total_neurons
    top_percent_in_benign = 100 * rank_in_benign / num_total_neurons
    
    print('{}-{}'.format(layer, neuron))
    print('Top %.2lf%% in benign (White wolf) --> top %.2lf%% in attacked (Mexican hairless)' % (top_percent_in_benign, top_percent_in_attacked))
    
    plt.figure()
    
    # Feature vis
    plt.subplot(231)
    feature_vis_path = '{}/channel/{}-{}-channel.jpg'.format(vis_dirpath, layer, neuron)
    feature_vis = plt.imread(feature_vis_path)
    plt.axis('off')
    plt.imshow(feature_vis)
    
    # Example patch 0
    plt.subplot(234)
    ex0_path = '{}/dataset-p/{}-{}-dataset-p-0.jpg'.format(vis_dirpath, layer, neuron)
    ex0 = plt.imread(ex0_path)
    plt.axis('off')
    plt.imshow(ex0)
    
    # Example patch 1
    plt.subplot(232)
    ex1_path = '{}/dataset-p/{}-{}-dataset-p-1.jpg'.format(vis_dirpath, layer, neuron)
    ex1 = plt.imread(ex1_path)
    plt.axis('off')
    plt.imshow(ex1)
    
    # Example patch 2
    plt.subplot(235)
    ex2_path = '{}/dataset-p/{}-{}-dataset-p-2.jpg'.format(vis_dirpath, layer, neuron)
    ex2 = plt.imread(ex2_path)
    plt.axis('off')
    plt.imshow(ex2)
    
    # Example patch 3
    plt.subplot(233)
    ex3_path = '{}/dataset-p/{}-{}-dataset-p-3.jpg'.format(vis_dirpath, layer, neuron)
    ex3 = plt.imread(ex3_path)
    plt.axis('off')
    plt.imshow(ex3)

    # Example patch 4
    plt.subplot(236)
    ex4_path = '{}/dataset-p/{}-{}-dataset-p-4.jpg'.format(vis_dirpath, layer, neuron)
    ex4 = plt.imread(ex4_path)
    plt.axis('off')
    plt.imshow(ex4)
    
    plt.show()


In [ ]:
print('Look into neurons that are no longer activated in attacked image, while was very activated by benign image.')
print('The attacked images lose the following features.')
print('Neurons:', both_activated_neurons[:topk])
print()

vis_dirpath = '/Users/haekyu/data/summit/summit-data/data/feature-vis'
for neuron in both_activated_neurons[:topk]:
    
    rank_in_attacked = np.argwhere(rank_of_neurons_attacked == neuron)
    rank_in_benign = np.argwhere(rank_of_neurons_benign == neuron)
    top_percent_in_attacked = 100 * rank_in_attacked / num_total_neurons
    top_percent_in_benign = 100 * rank_in_benign / num_total_neurons
    
    print('{}-{}'.format(layer, neuron))
    print('Top %.2lf%% in benign (White wolf) --> top %.2lf%% in attacked (Mexican hairless)' % (top_percent_in_benign, top_percent_in_attacked))
    
    plt.figure()
    
    # Feature vis
    plt.subplot(231)
    feature_vis_path = '{}/channel/{}-{}-channel.jpg'.format(vis_dirpath, layer, neuron)
    feature_vis = plt.imread(feature_vis_path)
    plt.axis('off')
    plt.imshow(feature_vis)
    
    # Example patch 0
    plt.subplot(234)
    ex0_path = '{}/dataset-p/{}-{}-dataset-p-0.jpg'.format(vis_dirpath, layer, neuron)
    ex0 = plt.imread(ex0_path)
    plt.axis('off')
    plt.imshow(ex0)
    
    # Example patch 1
    plt.subplot(232)
    ex1_path = '{}/dataset-p/{}-{}-dataset-p-1.jpg'.format(vis_dirpath, layer, neuron)
    ex1 = plt.imread(ex1_path)
    plt.axis('off')
    plt.imshow(ex1)
    
    # Example patch 2
    plt.subplot(235)
    ex2_path = '{}/dataset-p/{}-{}-dataset-p-2.jpg'.format(vis_dirpath, layer, neuron)
    ex2 = plt.imread(ex2_path)
    plt.axis('off')
    plt.imshow(ex2)
    
    # Example patch 3
    plt.subplot(233)
    ex3_path = '{}/dataset-p/{}-{}-dataset-p-3.jpg'.format(vis_dirpath, layer, neuron)
    ex3 = plt.imread(ex3_path)
    plt.axis('off')
    plt.imshow(ex3)

    # Example patch 4
    plt.subplot(236)
    ex4_path = '{}/dataset-p/{}-{}-dataset-p-4.jpg'.format(vis_dirpath, layer, neuron)
    ex4 = plt.imread(ex4_path)
    plt.axis('off')
    plt.imshow(ex4)
    
    plt.show()


In [ ]:
neuron = got_activated_neurons[0]
prev_layer = args.layers[args.layers.index(layer) - 1]

with tf.Graph().as_default():
    
    # Define activation render
    t_input = tf.placeholder(tf.float32, [None, 224, 224, 3])
    googlenet.import_graph(t_input)
    
    t_w0, t_w1, t_w2, t_w3, t_w4, t_w5 = model_helper.get_weight_tensors(layer)
    t_a0, t_a1, t_a2 = model_helper.get_intermediate_layer_tensors(prev_layer, layer)
    t_curr_act_map = tf.get_default_graph().get_tensor_by_name('import/%s:0' % layer)
    
    blk = model_helper.get_blk_of_neuron(args, layer, neuron)
    
    if blk == 0:
        t_a = t_a0
        t_w = t_w0
        prev_name = prev_layer
    elif blk == 1:
        t_a = t_a1
        t_w = t_w2
        prev_name = '%s_3x3_bottleneck:0' % layer
    elif blk == 2:
        t_a = t_a2
        t_w = t_w4
        prev_name = '%s_5x5_bottleneck:0' % layer
    elif blk == 3:
        t_a = t_a0
        t_w = t_w5
        prev_name = prev_layer
    
    with tf.Session() as sess:
        prev_act_maps, weight_filter, curr_act_maps = sess.run([t_a, t_w, t_curr_act_map], feed_dict={t_input: [raw_attacked]})

In [ ]:
prev_act_map = prev_act_maps[0]
curr_act_map = curr_act_maps[0]
prev_act_map_grayscale = np.average(prev_act_map, axis=2)
curr_act_map_grayscale = np.average(curr_act_map, axis=2)

In [ ]:
print('Activation map in the previous layer ({})'.format(prev_name))

plt.figure()

# Attacked image
plt.subplot(1, 3, 1)
plt.axis('off')
plt.imshow(raw_attacked)
plt.text(25, -10, 'Attacked image')

# Previous layer's activation map
prev_act_map_grayscale = np.average(prev_act_map, axis=2)
plt.subplot(1, 3, 2)
plt.axis('off')
plt.imshow(prev_act_map_grayscale)
plt.text(0, -2.8, 'Activation map avg')
plt.text(1, -1.2, 'in previous layer')

# Example patch 0
plt.subplot(1, 3, 3)
plt.axis('off')
plt.imshow(curr_act_map_grayscale)
plt.text(0, -2.8, 'Activation map avg')
plt.text(1, -1.0, 'in current layer')

plt.show()

In [ ]:
neuron = got_activated_neurons[0]
neuron_th = neuron
for b in range(blk - 1, -1, -1):
    b_size = args.layer_blk_sizes['{}_{}'.format(layer, b)]
    neuron_th -= b_size
    
print('{} in {}, {} in block {}'.format(neuron, layer, neuron_th, blk))

num_prev_neurons = prev_act_map.shape[-1]
num_rows = num_prev_neurons

plt.figure(figsize=(10, 200))
for prev_neuron in range(num_prev_neurons):
    if prev_neuron + 1 == num_rows:
        break
        
    prev_neuron_act_map = prev_act_map[:, :, prev_neuron]
    plt.subplot(num_rows, 3, 3 * (prev_neuron + 1) + 1)
    plt.axis('off')
    plt.imshow(prev_neuron_act_map, cmap='gray')
    plt.text(0, -2, 'prev_{}'.format(prev_neuron))
    plt.colorbar()
    
    w = weight_filter[:, :, prev_neuron, neuron_th]
    plt.subplot(num_rows, 3, 3 * (prev_neuron + 1) + 2)
    plt.axis('off')
    plt.imshow(w, cmap='gray')
    plt.text(0, -2, 'filter_{}'.format(prev_neuron))
    plt.colorbar()
    
    f = cv2.filter2D(prev_neuron_act_map, -1, w)
    plt.subplot(num_rows, 3, 3 * (prev_neuron + 1) + 3)
    plt.axis('off')
    plt.imshow(f, cmap='gray')
    plt.text(0, -2, 'depth_{}'.format(prev_neuron))
    plt.colorbar()
    
plt.show()